In [ ]:
# 라이브러리 import
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import clone
import matplotlib
import warnings

In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터 로드
URL = "/content/drive/MyDrive/SIL/WORK/DSS 머신러닝2/" # 데이터 위치 설정
data = pd.read_csv(URL+"data/class_balance.csv",encoding="EUC-KR")

In [ ]:
data['Y'] = data['Y'].map(lambda x: 0 if x == -1 else 1) # 기존 -1, 1이었던 Y값을 모델 학습을 위해 0, 1로 변경

In [ ]:
# X, Y 분할
Y=data["Y"].copy()
X=data.drop("Y",axis=1)
X.head(3)

In [ ]:
# 평가 지표 출력 함수
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix

def print_all_reg(Y_test,pred):
    # Specificity를 구하기 위해 confusion matrix를 이용
    cm1 = confusion_matrix(Y_test,pred)
    specificity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])

    #결과 검사
    #recall = cm1[1,1]/(cm1[1,1]+cm1[1,0])
    #pre = cm1[1,1]/(cm1[1,1]+cm1[0,1])

    G_mean = recall_score(Y_test,pred) * specificity1

    print("model의 recall 값은 {:.3f}".format(recall_score(Y_test,pred)))
    print("model의 2종 오류 확률 값은 {:.3f}".format(1-recall_score(Y_test,pred)))
    print("model의 Specificity 값은 {:.3f}".format(specificity1))
    print("model의 1종 오류 확률 값은 {:.3f}".format(1-specificity1))
    print("model의 precision 값은 {:.3f}".format(precision_score(Y_test,pred)))
    print("model의 f1_score 값은 {:.3f}".format(f1_score(Y_test,pred)))
    print("model의 G-mean 값은 {:.3f}".format(np.sqrt(G_mean)))
    print("model의 accuracy 값은 {:.3f}".format(accuracy_score(Y_test,pred)))

## Blending
- Blending은 Train_set을 통해 학습한 모델로 validation_set을 예측하고, validationset의 원본 X데이터와 validation_set에 대한 예측값을 모두 입력변수로 활용하여 test_set을 예측함
- 따라서 충분한 양의 데이터가 확보가 되지 않은 경우 적은양의 validation 데이터로 학습을 진행하므로 과적합에대한 우려가 높음(일반화성능이 낮을 우려가 있음)


In [ ]:
# 데이터셋을 학습 세트, 검증 세트, 테스트 세트로 분리
X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22, shuffle=True)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_temp, Y_temp, test_size=0.25, random_state=22, shuffle=True)

# 특성 스케일링 (선택 사항)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
# X_temp = scaler.fit_transform(X_temp)
X_test = scaler.transform(X_test)


# 일반 모델의 평가 방법
normal_model = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss')
normal_model.fit(X_train, Y_train)

# 최종 예측
normal_pred = normal_model.predict(X_test)

# 성능 평가
print(f'\nNormal Model Performance:')
print_all_reg(Y_test, normal_pred)
# 데이터셋을 학습 세트, 검증 세트, 테스트 세트로 분리
X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22, shuffle=True)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_temp, Y_temp, test_size=0.25, random_state=22, shuffle=True)

# 특성 스케일링 (선택 사항)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# 모델 정의
models = {
    'lr':LogisticRegression(max_iter=1000, C=1.0, solver='lbfgs'),
    'svc': SVC(probability=True, C=1.0, kernel='rbf', gamma='scale'),
    'rf': RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1,random_state=57),

}

val_preds = []
test_preds = []

# 모델 학습 및 예측
for name, model in models.items():
    model.fit(X_train, Y_train)
    val_pred = pd.DataFrame(model.predict(X_valid), columns=[f'pred_{name}'])
    test_pred = pd.DataFrame(model.predict(X_test), columns=[f'pred_{name}'])
    val_preds.append(val_pred)
    test_preds.append(test_pred)

# 원래 feature와 예측값을 결합하여 새로운 데이터셋 생성
df_val = pd.concat([pd.DataFrame(X_valid)] + val_preds, axis=1)
df_test = pd.concat([pd.DataFrame(X_test)] + test_preds, axis=1)

# 모든 컬럼 이름을 문자열로 변환
df_val.columns = df_val.columns.astype(str)
df_test.columns = df_test.columns.astype(str)

# 메타 모델 정의 및 학습
meta_model = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=1)
meta_model.fit(df_val, Y_valid)

# 최종 예측
final_pred = meta_model.predict(df_test)

# 성능 평가
print(f'\nBlending Model Performance:')
print_all_reg(Y_test, final_pred)


Normal Model Performance:
model의 recall 값은 0.611
model의 2종 오류 확률 값은 0.389
model의 Specificity 값은 0.970
model의 1종 오류 확률 값은 0.030
model의 precision 값은 0.917
model의 f1_score 값은 0.733
model의 G-mean 값은 0.770
model의 accuracy 값은 0.843

Blending Model Performance:
model의 recall 값은 0.778
model의 2종 오류 확률 값은 0.222
model의 Specificity 값은 0.970
model의 1종 오류 확률 값은 0.030
model의 precision 값은 0.933
model의 f1_score 값은 0.848
model의 G-mean 값은 0.868
model의 accuracy 값은 0.902


In [ ]:
df_val.head(5)

,0,1,2,3,4,5,6,7,8,9,...,467,468,469,470,471,472,473,pred_lr,pred_svc,pred_rf
0,-2.164327,0.445034,-0.805535,0.774500,0.320196,-0.837607,0.944401,0.048515,0.089229,-0.339936,...,-0.160916,-0.158886,-0.159310,0.121719,-0.327607,-0.295582,-0.516985,0,0,0
1,-0.667285,0.510772,0.485514,0.669279,0.657029,2.412248,-0.460567,-0.164716,-0.778083,0.658169,...,-0.205912,-0.207552,-0.202083,-0.531691,-0.339042,-0.409824,-0.144709,1,1,0
2,0.561932,2.555113,-0.046328,-0.244252,-0.049102,-0.093991,0.411482,1.689744,1.229711,-0.613102,...,-0.176797,-0.183219,-0.174360,-0.041634,0.404229,0.656428,-0.060217,0,0,0
3,-0.986188,-0.416421,-0.880469,-1.013078,-1.362748,-0.180070,-0.266778,-1.733253,-0.054185,-1.737284,...,-0.060337,-0.073720,-0.067440,-1.415717,-0.133213,0.085222,2.105125,0,0,0
4,0.807804,2.499189,1.348161,-0.965805,-1.999888,1.194183,-1.720193,0.437822,1.188736,-0.098290,...,-0.203265,-0.134553,-0.197270,-0.012807,0.152660,0.199463,-0.212454,0,0,0


In [ ]:
df_test.head(5)

,0,1,2,3,4,5,6,7,8,9,...,467,468,469,470,471,472,473,pred_lr,pred_svc,pred_rf
0,1.245984,2.031627,-0.804801,-1.111996,-0.715867,0.196442,0.120799,-2.561944,0.041425,-0.098290,...,-0.160916,-0.098053,-0.154002,-0.051243,0.644362,0.656428,0.064307,0,0,0
1,1.039188,-1.512866,0.461387,0.681569,1.008065,-0.769696,0.072352,-0.219639,-0.634669,2.822482,...,-0.076218,-0.073720,-0.075492,-0.973704,0.461403,0.351784,1.173952,1,0,1
2,-1.282991,-0.494124,-0.723291,1.006697,1.411452,-0.934643,0.314588,-1.240561,0.082400,-1.096395,...,-0.229733,-0.195386,-0.223140,-0.493256,1.033150,1.760760,0.750787,1,1,1
3,-0.304623,-0.366142,3.531465,3.115299,-0.630239,-3.347959,-2.931372,-0.302023,0.601422,2.087036,...,-0.203265,-0.207552,-0.197245,-0.502864,-1.516839,-1.552236,-0.973674,1,1,1
4,-2.281263,-0.576263,-0.171342,-0.947814,-1.128183,0.649625,0.217694,0.142207,-0.409305,0.542599,...,-0.237673,-0.207552,-0.229515,1.322841,-0.750699,-0.981030,-0.944731,1,0,0
